In [ ]:
%env NUMBA_ENABLE_CUDASIM 1

In [ ]:
import h5py
import numpy as np
import numba
import matplotlib
from matplotlib.pyplot import figure, colorbar, savefig, title, xlabel, ylabel, imshow, close
import timeit
import math
from numba import cuda
import llvmlite.binding as llvm
from pdb import set_trace;
llvm.set_option('', '--debug-only=loop-vectorize')

In [ ]:
@numba.jit(nopython=True, nogil=True, cache=True)
def onemove_in_cube_true_numba(p0, v):
    htime = np.abs((np.floor(p0) - p0 + (v > 0)) / v) 
    print(htime)
    minLoc = np.argmin(htime) 
    dist = htime[minLoc] 
    htime = p0 + dist * v 
    htime[minLoc] = np.round(htime[minLoc]) + np.spacing(np.abs(htime[minLoc])) * np.sign(v[minLoc])
    return htime, dist

In [ ]:
a,b=onemove_in_cube_true_numba(np.array([104,128,0]), np.array([-0.70710677, -0.70710677, 1e-16]))

In [ ]:
p0=np.array([1.1,2.2,33.3])
v

In [ ]:
def main_loop(Nx, Ny, Nz, Mx, My, D, h, orginOffset, ep, mu):
    detector = np.zeros((Mx, My), dtype=np.float32) 
    for i in range(Mx): 
        for j in range(My):
            pos = np.array([orginOffset[0] + i * D, orginOffset[1] + D * j, 0], dtype=np.float32) 
            dir = ((ep - pos) / np.linalg.norm(ep - pos)).astype(np.float32)
            dir[dir == 0] = 1e-16
            L = 0  
            h_z = h + Nz
            while pos[2] < h_z:  
                print(pos, dir)
                pos, dist = onemove_in_cube_true_numba(pos, dir)  
                if 0 <= pos[0] < Nx and 0 <= pos[1] < Ny and h <= pos[2] < h_z:
                    L += mu[int(np.floor(pos[0])),int(np.floor(pos[1])) , int(np.floor(pos[2] - h))] * dist
            detector[i][j] = L 
    return detector

In [ ]:
f = h5py.File('headct.h5', 'r') #HDF5 file containing Headct array of linear attenuation coeffcients(Nx,Ny,Nz)
headct=np.array(f.get('ct'))
headct=np.transpose(headct) #linear attenuation coeffceient matrix
det=f.get('det')
det=np.transpose(det) #Result of running this code on Matlab for later comparsion
Nx = np.size(headct,0) #Imaging x dimension length in mm
Ny = np.size(headct,1) #Imaging y dimension length in mm
Nz = np.size(headct,2) #Imaging z dimension length in mm
Mx = 200 #Number of pixels in x direction
My = 200 #Number of pixels in y direction
D = 2 #Size of each pixel in mm
h = 50 #distance(Z) bettween bottom of imaging volume and detector
H = h + Nz + 200 #distance(Z) bettween detector and x-ray source
muBone = 0.573 #linear attenuation coeffcient bone cm^-1
muFat = 0.193 #linear attenuation coeffcient fat cm^-1
orginOffset = np.array([(-Mx * D) / 2 + (Nx / 2), (-My * D) / 2 + (Ny / 2), 0], dtype=np.float32) #offset from origin to detector start (X,Y,Z)
ep = np.array([Nx / 2, Ny / 2, H], dtype = np.float32) #location of x-ray soruce
orginOffset = np.array([(-Mx * D) / 2 + (Nx / 2), (-My * D) / 2 +(Ny / 2), 0],dtype=np.float32) #offset from origin to detector start
mu=np.zeros((Nx,Ny,Nz),dtype=np.float32) #(Nx,Ny,Nz) linear attenuation coeffcient matrix 
mu[np.nonzero(headct>0)]=((headct[np.nonzero(headct>0)]-0.3)/(0.7))*(muBone-muFat)+muFat #Normilization of givens mus of linear attenuation matrix

In [ ]:
main_loop_par=numba.njit(main_loop)

In [ ]:
detA = main_loop_par(Nx, Ny, Nz, Mx, My, D, h, orginOffset, ep, mu)
detector = np.exp(detA * -10, dtype=np.float64)

In [ ]:
imshow(np.log(detector))

In [ ]:
@cuda.jit(device=True)
def onemove_in_cube_true_numba(p0, v):
    htime=cuda.local.array((3), dtype=numba.float32)
    for i in range(3):
        if v[i]>0:
            htime[i]=abs((math.floor(p0[i]) - p0[i] + 1) / v[i])  
        else:
            htime[i]=abs((math.floor(p0[i]) - p0[i]) / v[i])  

    minA=0
    minV=htime[0]
    for i in range(1,3):
        if minV>htime[i]:
            minA=i
            minV=htime[i]
    dist = htime[minA] 
    for i in range(3):
        htime[i] = p0[i] + dist * v[i] 
    if v[minA]<0:
        htime[minA] = round(htime[minA]) + 1.5e-19 * -1
    else:
        htime[minA] = round(htime[minA]) + 1.5e-19 
    return htime, dist

In [ ]:
@cuda.jit
def main_loop(obj_dim, scene_info, orginOffset, ep, mu, detector):
    i, j = cuda.grid(2)
    if i < detector.shape[0] and j <detector.shape[1]:
        pos=cuda.local.array((3), dtype=numba.float32)
        direction=cuda.local.array((3), dtype=numba.float32)
        pos[0]=orginOffset[0] + i * scene_info[0]
        pos[1]=orginOffset[1] + scene_info[0] * j
        pos[2]=0
        norm=0
        for k in range(3):
            norm+=math.pow(ep[k]-pos[k],2)  
        norm=math.sqrt(norm)
        
        for k in range(3):
            direction[k]=(ep[k]-pos[k])/norm
            if direction[k]==0:
                direction[k]=1e-16        
        L = 0  
        h_z = scene_info[1] + obj_dim[1]
        while pos[2] < h_z:  
            pos, dist = onemove_in_cube_true_numba(pos, direction)
            print(pos)
            if 0 <= pos[0] < obj_dim[0] and 0 <= pos[1] < obj_dim[1] and  scene_info[1] <= pos[2] < h_z:
                L += mu[int(math.floor(pos[0])),int(math.floor(pos[1]) ),int( math.floor(pos[2] -  scene_info[1]))] * dist
        detector[i][j] = L 


In [ ]:
stream = cuda.stream()
h_detector = np.zeros((Mx, My), dtype=np.float32) 
d_detector = cuda.to_device(h_detector, stream)
d_mu=cuda.to_device(mu)
h_obj_dim=np.array([Nx,Ny, Nz])
d_obj_dim = cuda.to_device(h_obj_dim, stream)
h_scene_info=np.array([D, h])
d_scene_info = cuda.to_device(h_scene_info, stream)
d_ep = cuda.to_device(orginOffset, stream)
d_orginOffset = cuda.to_device(ep, stream)
main_loop[10, 10, stream](d_obj_dim, d_scene_info, d_orginOffset, d_ep, d_mu, d_detector)
detector = d_detector.copy_to_host(stream=stream)

In [ ]:
i=1
j=1
pos=[0,  0, 0]
pos[0]=d_orginOffset[0] + i * d_scene_info[0]
pos[1]=d_orginOffset[1] + d_scene_info[0] * j
pos[2]=0

In [ ]:
res=np.array(pos)

In [ ]:
np.floor(res)-res